## ------------------------------------- XGBoost - SHAP ---------------------------------

In [ ]:
### -------------------------------------------
import pandas as pd
import xgboost
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pandas import DataFrame
import sys
import warnings
import os
warnings.filterwarnings(action='ignore')
zh = DataFrame(np.arange(1).reshape(1,1))

cohort = 4
dataDir = r'datapath'
train_filename = '/train.csv'
test_filename = '/test.csv'
val_filename = '/val.csv'
val2_filename = '/val.csv'

## --------------------------------------------
train_data = pd.read_csv(dataDir + train_filename)
print(train_data.iloc[0:3, 0:5])
print('\n', train_data.shape)        

In [ ]:
cc =2
x_train = train_data.iloc[:,cc:]
y_train = train_data.iloc[:,1]    
ratio = 1- sum(y_train == 1) / len(y_train) 
print('\n', round(ratio, 3))
print(x_train.iloc[0:1, 0:3], '\n')
print('\n', x_train.shape)

## ---------------------------------------
test_data = pd.read_csv(dataDir + test_filename)
x_test = test_data.iloc[:,cc:]
y_test = test_data.iloc[:,1] 
print(x_test.iloc[0:1, 0:3], '\n')
print('\n', x_test.shape)

## --------------------------------------
val_data = pd.read_csv(dataDir + val_filename)
x_val = val_data.iloc[:,cc:]
y_val = val_data.iloc[:,1] 
print(x_val.iloc[0:1, 0:3])
print('\n', x_val.shape)

## ---------------------------------------
val2_data = pd.read_csv(dataDir + val2_filename)
x_val2 = val2_data.iloc[:,cc:]
y_val2 = val2_data.iloc[:,1] 
print(x_val2.iloc[0:1, 0:3])
print('\n', x_val2.shape)

In [ ]:
# ------------------------------------
import time
format_date = time.strftime("Model_%Y-%m-%d_%H.%M", time.localtime(time.time()))
print(dataDir + '/' + format_date)

import os
os.mkdir(dataDir + '/' + format_date)

# ------------------------------------- XGBoost  ----------------------------------

In [ ]:
model = 'Model_Xgboost'
best_score = 0
print('Baseline:', best_score)
from xgboost import XGBClassifier

for n_estimators in range(3, 16, 3):
    for max_depth in range(1, 4, 1):
        for colsample_bytree in [0.5]:
            for subsample in [0.6, 0.4]:
                for child in [1]:
                    model_forest = XGBClassifier(max_depth=max_depth, colsample_bytree=colsample_bytree, n_estimators=n_estimators, subsample=subsample, booster='gbtree',
                                                 child=child, gpu_id = 0, n_jobs=-1, random_state=60).fit(x_train, y_train)
                    from sklearn import metrics
                    from sklearn.metrics import roc_auc_score
                    train_score = round(metrics.roc_auc_score(y_train, model_forest.predict_proba(x_train)[:,1]), 3)
                    test_score = round(metrics.roc_auc_score(y_test, model_forest.predict_proba(x_test)[:,1]), 3)
                    val_score = round(metrics.roc_auc_score(y_val, model_forest.predict_proba(x_val)[:,1]), 3)
                    val2_score = round(metrics.roc_auc_score(y_val2, model_forest.predict_proba(x_val2)[:,1]), 3)

In [ ]:
import joblib
print(dataDir + '\\' + format_date + '\\' + '  model_Xgboost.pkl')
joblib.dump(best_model, dataDir + '\\' + format_date + '\\' +'Model_Xgboost.pkl')

In [ ]:
# ----------------------------forest--------------------------------------
import joblib
format_date = 'Model_2023-06-14_23.30'
print(dataDir + '\\' + format_date + '\\' + '  Model_Xgboost.pkl')
joblib.dump(best_model, dataDir + '\\' + format_date + '\\' +'Model_Xgboost.pkl')

In [ ]:
import joblib
model_file = dataDir + '\\' + format_date + '\\' + model + '.pkl'
best_model = joblib.load(model_file)
print(best_model)

In [ ]:
plt.close(); plt.close(); plt.close(); plt.close()
import seaborn as sns
f_importance = best_model.feature_importances_
head_lst = x_train.columns.values.tolist()
print(head_lst)
print(f_importance)
df_importance = pd.DataFrame({'head':head_lst,'importance':f_importance})
df_importance = df_importance.sort_values(by=['importance'],ascending=False) 

df_p_i = df_importance[df_importance['importance']>0.00000]
myfig = plt.gcf()
plt.gcf().set_size_inches(4,4)
sns.barplot(x='importance',y='head',data=df_p_i)
plt.xlabel('Feature importance of ' + model, fontsize=10)
plt.ylabel('')
myfig.savefig(dataDir + '\\' + format_date + '\\' + '1.Importance.pdf',dpi=350,bbox_inches = 'tight')

In [ ]:
from sklearn import metrics
import seaborn as sns
def model_perform(x, y, savename, cutoff):
    y_proba = best_model.predict_proba(x)[:, 1]
    print(y_proba)
    if cutoff == 'Train':
        fpr, tpr, threshold = metrics.roc_curve(y, y_proba, pos_label=1)
        cutoff = threshold[np.argmax(tpr-fpr)]
    y_predict = np.int64(y_proba >= cutoff)
    
    Matrix = metrics.confusion_matrix(y, y_predict)
    
    print('\n', Matrix)
    print('Accuracy   ',  "%.3f" % ((Matrix[1,1]+Matrix[0,0])/sum(sum(Matrix))))
    print('Sensitivity',  "%.3f" % (Matrix[1,1]/(Matrix[1,1]+Matrix[1,0])))
    print('Specificity',  "%.3f" % (Matrix[0,0]/(Matrix[0,1]+Matrix[0,0])))
    print('PPV        ',  "%.3f" % (Matrix[1,1]/(Matrix[1,1]+Matrix[0,1])))
    print('NPV        ',  "%.3f" % (Matrix[0,0]/(Matrix[0,0]+Matrix[1,0])))
    print('F1 score   ',  "%.3f" % (2/(2+(Matrix[1,0]+Matrix[0,1])/Matrix[1,1])))
    print('G-mean     ',  "%.3f" % pow((Matrix[1,1]/(Matrix[1,1]+Matrix[0,1]))*(Matrix[0,0]/(Matrix[0,1]+Matrix[0,0])), 0.5))
    print('AUC Score  ',  "%.3f" % (metrics.roc_auc_score(y, y_proba)))
    print('\n')
    
    df = x
    df['label'] = y
    df['probably'] = y_proba
    df.to_csv(dataDir + '\\' + format_date + '\\' + savename, encoding='gbk', index=False)
    return cutoff

cutoff = model_perform(train_data.iloc[:,cc:], y_train, model + '_train_predict.csv', 'Train')
model_perform(test_data.iloc[:,cc:], y_test, model + '_test_predict.csv', cutoff)
model_perform(val_data.iloc[:,cc:], y_val, model + '_val_predict.csv', cutoff)
model_perform(val2_data.iloc[:,cc:], y_val2, model + '_val2_predict.csv', cutoff)
print('Finished')

# ------------------------------ SHAP -----------------------------

In [ ]:
import shap
shap.initjs()
explainer = shap.TreeExplainer(best_model)
shap_values = explainer(x_train)
sum_shap_values = explainer.shap_values(x_train)

In [ ]:
# ---------------------------- SHAP Beeswarm plot ----------------------------
Max_display = df_p_i.shape[0]
import matplotlib.pyplot as plt
def SHAP_bar(x, Max_display, filename, width, height):
    myfig = plt.gcf()
    sum_shap_values = explainer.shap_values(x)
    shap.summary_plot(sum_shap_values, x, max_display=Max_display)
    myfig.savefig(dataDir + '\\' + format_date + '\\' + filename + ' .pdf', dpi=300, bbox_inches = 'tight')
    myfig.savefig(dataDir + '\\' + format_date + '\\' + filename + '.tiff', dpi=300, bbox_inches = 'tight')

SHAP_bar(x_train, Max_display, '2. Beeswarm_train', 2, 4)
SHAP_bar(x_test, Max_display,  '2. Beeswarm_test', 2, 4)
SHAP_bar(x_val,  Max_display,  '2. Beeswarm_val', 6, 4)
SHAP_bar(x_val2, Max_display,  '2. Beeswarm_val2', 6, 4)

In [ ]:
x = x_train; y = y_train; data = train_data
import shap; shap.initjs(); explainer = shap.TreeExplainer(best_model); shap_values = explainer(x); sum_shap_values = explainer.shap_values(x)
print(x.shape[0])

In [ ]:
# ------- SHAP -------
import os
if not os.path.exists(os.path.join(dataDir,"waterfallplot")):
    os.makedirs(os.path.join(dataDir,"waterfallplot"))
for i in range(0, int(x.shape[0]/10), 1):
    shap_values = explainer(x)
    shap_values.values = np.round(shap_values.values,3)
    shap_values.base_values = np.round(shap_values.base_values,3)
    shap_values.data = np.round(shap_values.data,3)
    PP = i
    model_p = round(best_model.predict_proba(x)[PP, 1] ,3)
    print(PP, y[i], model_p)
    shap.initjs()
    plt.figure(figsize=(6,6))
    shap.plots.waterfall(shap_values[PP], show = False, max_display = 10)
    plt.savefig(os.path.join(os.path.join(dataDir,'waterfallplot'), 'SHAP_ID={},Lable={},probably={}.pdf'.format(str(data.iloc[:,0][PP]),str(y[PP]),str(model_p))), dpi=300,bbox_inches = 'tight',pad_inches=0.1) # DT
    print(model_p)

In [ ]:
if not os.path.exists(os.path.join(dataDir,"foreplot")):
    os.makedirs(os.path.join(dataDir,"foreplot"))
    print(os.path.join(dataDir,"foreplot"))
for i in range(0, 40, 1):
    shap_values = explainer(round(x, 3))
    PP = i
    model_p = round(best_model.predict_proba(x)[PP, 1], 3)
    print(data.iloc[PP, 0], y[PP], model_p)
    shap.initjs()
    shap.plots.force(explainer.expected_value,shap_values.values[PP],feature_names=list(x.columns),matplotlib=True, show=False, text_rotation = 7)
#     plt.savefig(os.path.join(os.path.join(dataDir,"foreplot"), "ID = {}, label = {}, p = {}.tiff".format(val_data.iloc[PP, 0], y_val[PP], str(model_p))), dpi=50,bbox_inches = 'tight')
    plt.savefig(os.path.join(os.path.join(dataDir,"foreplot"), "{}, p = {}.pdf".format(str(PP + 1), str(model_p))), dpi=300,bbox_inches = 'tight')